In [ ]:
#!python3 -m venv moje-prostredi

In [ ]:
#!./moje-prostredi/bin/pip install ipykernel

In [ ]:
#!./moje-prostredi/bin/python -m ipykernel install --user --name="moje-prostredi" --display-name="Python (moje-prostredi)"

In [ ]:
#!pip install rfdetr

In [ ]:
#!pip uninstall -y opencv-python opencv-python-headless

In [ ]:
#!pip install opencv-python-headless

In [ ]:
#!pip uninstall -y numpy scipy

In [ ]:
#!pip install numpy scipy

In [1]:
import torch
import json
import os
import time
from pathlib import Path
from rfdetr.detr import RFDETRMedium
import traceback

def get_optimal_batch_size():
    """Automaticky detekuje optimální batch size podle VRAM"""
    if not torch.cuda.is_available():
        return 2
    
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    if vram_gb >= 24:    # RTX 4090, A100
        return 8
    elif vram_gb >= 16:  # RTX 4080, RTX 3080 Ti
        return 6  
    elif vram_gb >= 12:  # RTX 4070 Ti, RTX 3080
        return 4
    elif vram_gb >= 8:   # RTX 4060 Ti, RTX 3070
        return 3
    else:               # RTX 3060, GTX 1660
        return 2

def validate_dataset(dataset_dir):
    """Validuje dataset strukturu"""
    dataset_path = Path(dataset_dir)
    
    if not dataset_path.exists():
        raise FileNotFoundError(f"Dataset složka neexistuje: {dataset_dir}")
    
    required_splits = ['train']
    optional_splits = ['valid', 'test']
    
    found_splits = []
    for split in required_splits + optional_splits:
        split_dir = dataset_path / split
        ann_file = split_dir / "_annotations.coco.json"
        
        if split_dir.exists() and ann_file.exists():
            with open(ann_file, 'r') as f:
                data = json.load(f)
            
            n_images = len(data['images'])
            n_annotations = len(data['annotations'])
            n_categories = len(data['categories'])
            
            found_splits.append(split)
            print(f"✅ {split}: {n_images} obrázků, {n_annotations} anotací, {n_categories} kategorií")
            
            print(f"📋 Kategorie v {split}:")
            for cat in data['categories']:
                count = sum(1 for ann in data['annotations'] if ann['category_id'] == cat['id'])
                print(f"  ID {cat['id']}: {cat['name']} - {count} objektů")
            
            crowd_count = sum(1 for ann in data['annotations'] if ann.get('iscrowd', 0) == 1)
            if crowd_count > 0:
                print(f"⚠️  {split}: {crowd_count} 'crowd' objektů - budou ignorovány!")
        else:
            if split in required_splits:
                raise FileNotFoundError(f"Chybí {split} split: {split_dir}")
            else:
                print(f"ℹ️  {split}: není k dispozici")
    
    if not found_splits:
        raise ValueError("Žádné platné splits nenalezeny!")
    
    return found_splits

def check_rfdetr_fixes():
    """Zkontroluje, zda jsou aplikovány RF-DETR opravy"""
    try:
        import rfdetr
        detr_file = Path(rfdetr.__file__).parent / "detr.py"
        
        if not detr_file.exists():
            print("⚠️  Nelze najít RF-DETR detr.py soubor")
            return False
        
        with open(detr_file, 'r') as f:
            content = f.read()
        
        if "max([category['id'] for category in anns[\"categories\"]])" in content:
            print("✅ RF-DETR Fix 1: Single-class category bug opraven")
        else:
            print("❌ RF-DETR Fix 1: CHYBÍ single-class oprava!")
            print("   Musíš ručně opravit detr.py - viz návod v SETUP_GUIDE.md")
            return False
        
        if "num_classes + 1" in content:
            print("✅ RF-DETR Fix 2: Background class opraven")
        else:
            print("❌ RF-DETR Fix 2: CHYBÍ background class oprava!")
            return False
        
        return True
        
    except Exception as e:
        print(f"⚠️  Nelze zkontrolovat RF-DETR opravy: {e}")
        return False

NOTE! Installing ujson may make loading annotations faster.


In [4]:
# --- Zde nastavte parametry trénování ---

# Dataset a output
dataset_path = 'dataset'  # Path to your 80/10/10 split folder
output_path = 'output'

epochs = 100            # Standard for 2000 images
batch_size = None       # Keep as None so the script picks the best for your 2 GPUs
lr = 1e-5            # Your main learning rate
lr_encoder = 1e-6    # Add this line! (Typical value is 1/10th of lr)
weight_decay = 1e-4  # Ensure this is also defined
num_workers = 4      # Ensure this is also defined
grad_accum = 2          # Effective batch size of 16 (assuming batch 8 * 2)
use_ema = True          # Highly recommended for final accuracy

# Hardware
device_setting = 'auto'  # 'auto', 'cuda', 'cpu'
num_workers = 4

# Advanced
no_early_stopping = False
patience = 5
warmup_epochs = 2
grad_accum = 1

# Data augmentation
augment = False
mixup = 0.0
dropout = 0.1

# Debug
quick_test = False
skip_validation = False

# --- Konec nastavení ---

In [5]:
def train_in_notebook():
    print("🚀 RF-DETR Universal Training Script")
    print("=" * 50)
    
    # Validace
    if not skip_validation:
        print("\n🔍 Kontroluji dataset...")
        try:
            found_splits = validate_dataset(dataset_path)
        except Exception as e:
            print(f"❌ Dataset chyba: {e}")
            return
        
        print("\n🔧 Kontroluji RF-DETR opravy...")
        if not check_rfdetr_fixes():
            print("❌ RF-DETR není správně opraven! Viz SETUP_GUIDE.md")
            return
    
    # Device setup
    if device_setting == 'auto':
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    else:
        device = device_setting
    
    print(f"\n🖥️  Hardware:")
    print(f"  Device: {device}")
    
    if device == 'cuda':
        gpu_name = torch.cuda.get_device_name(0)
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"  GPU: {gpu_name}")
        print(f"  VRAM: {vram_gb:.1f} GB")
    
    # Batch size
    final_batch_size = batch_size
    if final_batch_size is None:
        final_batch_size = get_optimal_batch_size()
        print(f"  Auto batch size: {final_batch_size}")
    else:
        print(f"  Manual batch size: {final_batch_size}")
    
    effective_batch = final_batch_size * grad_accum
    print(f"  Effective batch size: {effective_batch}")
    
    final_epochs = 1 if quick_test else epochs
    if quick_test:
        print(f"\n⚡ Quick test mode: {final_epochs} epocha")

    # Model setup
    print(f"\n🤖 Inicializuji RF-DETR Medium...")
    model = RFDETRMedium()
    
    # Training config
    config = {
        'dataset_dir': dataset_path,
        'epochs': final_epochs,
        'batch_size': final_batch_size,
        'grad_accum_steps': grad_accum,
        'lr': lr,
        'lr_encoder': lr_encoder,
        'weight_decay': weight_decay,
        'num_workers': num_workers,
        'device': device,
        'output_dir': output_path,
        'early_stopping': not no_early_stopping,
        'early_stopping_patience': patience,
        'early_stopping_min_delta': 0.001,
        'warmup_epochs': warmup_epochs,
        'lr_drop': max(10, final_epochs // 3),
        'clip_max_norm': 0.1,
        'use_ema': True,
        'ema_decay': 0.995,
        'use_augment': augment,
        'mixup_alpha': mixup,
        'dropout_rate': dropout,
    }
    
    print(f"\n📋 Training konfigurace:")
    for key, value in config.items():
        print(f"  {key}: {value}")
    
    os.makedirs(output_path, exist_ok=True)
    
    config_file = Path(output_path) / "training_config.json"
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"\n💾 Konfigurace uložena: {config_file}")
    
    print(f"\n🎯 Spouštím training...")
    print(f"📁 Output: {output_path}")
    print(f"📊 Progress: tail -f {output_path}/train.log")
    print("-" * 50)
    
    try:
        model.train(**config)
        
        print("\n" + "=" * 50)
        print("🎉 Training dokončen!")
        print(f"📁 Model uložen v: {output_path}")
        
        checkpoint_best = Path(output_path) / "checkpoint_best_total.pth"
        if checkpoint_best.exists():
            print(f"🏆 Nejlepší model: {checkpoint_best}")
        
        print(f"\n📈 Pro inference použij:")
        print(f"  from rfdetr import RFDETRMedium")
        print(f"  model = RFDETRMedium.from_checkpoint('{checkpoint_best}')")
        print(f"  results = model.predict('image.jpg')")
        
    except KeyboardInterrupt:
        print("\n\n⏹️  Training přerušen uživatelem")
    except Exception as e:
        print(f"\n\n❌ Training chyba: {e}")
        traceback.print_exc()

# Spuštění trénování
train_in_notebook()

🚀 RF-DETR Universal Training Script

🔍 Kontroluji dataset...
✅ train: 1600 obrázků, 10360 anotací, 2 kategorií
📋 Kategorie v train:
  ID 1: lightbulb - 3955 objektů
  ID 2: sea_shell - 6405 objektů
✅ valid: 200 obrázků, 1245 anotací, 2 kategorií
📋 Kategorie v valid:
  ID 1: lightbulb - 491 objektů
  ID 2: sea_shell - 754 objektů
✅ test: 200 obrázků, 1345 anotací, 2 kategorií
📋 Kategorie v test:
  ID 1: lightbulb - 494 objektů
  ID 2: sea_shell - 851 objektů

🔧 Kontroluji RF-DETR opravy...
✅ RF-DETR Fix 1: Single-class category bug opraven
✅ RF-DETR Fix 2: Background class opraven

🖥️  Hardware:
  Device: cuda
  GPU: NVIDIA A40
  VRAM: 47.7 GB
  Auto batch size: 8
  Effective batch size: 8

🤖 Inicializuji RF-DETR Medium...
Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weight

fatal: not a git repository (or any of the parent directories): .git


Epoch: [0]  [  0/200]  eta: 0:12:19  lr: 0.000000  class_error: 73.22  loss: 9.4782 (9.4782)  loss_ce: 1.2426 (1.2426)  loss_bbox: 0.2857 (0.2857)  loss_giou: 0.3214 (0.3214)  loss_ce_0: 1.1724 (1.1724)  loss_bbox_0: 0.3506 (0.3506)  loss_giou_0: 0.3730 (0.3730)  loss_ce_1: 1.2253 (1.2253)  loss_bbox_1: 0.3152 (0.3152)  loss_giou_1: 0.3397 (0.3397)  loss_ce_2: 1.2310 (1.2310)  loss_bbox_2: 0.3133 (0.3133)  loss_giou_2: 0.3336 (0.3336)  loss_ce_enc: 1.1520 (1.1520)  loss_bbox_enc: 0.3954 (0.3954)  loss_giou_enc: 0.4271 (0.4271)  loss_ce_unscaled: 1.2426 (1.2426)  class_error_unscaled: 73.2194 (73.2194)  loss_bbox_unscaled: 0.0571 (0.0571)  loss_giou_unscaled: 0.1607 (0.1607)  cardinality_error_unscaled: 3879.0000 (3879.0000)  loss_ce_0_unscaled: 1.1724 (1.1724)  loss_bbox_0_unscaled: 0.0701 (0.0701)  loss_giou_0_unscaled: 0.1865 (0.1865)  cardinality_error_0_unscaled: 3856.0000 (3856.0000)  loss_ce_1_unscaled: 1.2253 (1.2253)  loss_bbox_1_unscaled: 0.0630 (0.0630)  loss_giou_1_unscaled: